# New York City Restaurant Inspection Results

## Exploration with Pandas

Data available at https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/xx67-kt59
or direct download at: https://data.cityofnewyork.us/api/views/xx67-kt59/rows.csv?accessType=DOWNLOAD

Data accessed June 11th, 2016

In [1]:
import pandas as pd

In [4]:
all_data = pd.read_csv("data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")
len(all_data)

450806

In [6]:
all_data.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40886675,CANARD INC,MANHATTAN,503,WEST 43 STREET,10036,2129471561,American,03/24/2014,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,17.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection
1,41480500,BLACK GOLD,BROOKLYN,461,COURT STREET,11231,3472278227,CafÃ©/Coffee/Tea,10/10/2013,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,38.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection


In [12]:
# any duplicates?
dups = all_data[all_data.duplicated()]
print( str(len(dups)) + " records are duplicated")

dups

6 records are duplicated


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
140713,41453018,ROCK BAR,MANHATTAN,185,CHRISTOPHER STREET,10014,2126751864,American,02/11/2015,Violations were cited in the following area(s).,10H,Proper sanitization not provided for utensil w...,Not Critical,23.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection
235500,41453018,ROCK BAR,MANHATTAN,185,CHRISTOPHER STREET,10014,2126751864,American,02/11/2015,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,23.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection
360241,41453018,ROCK BAR,MANHATTAN,185,CHRISTOPHER STREET,10014,2126751864,American,02/11/2015,Violations were cited in the following area(s).,22C,"Bulb not shielded or shatterproof, in areas wh...",Not Critical,NaN,NaN,NaN,06/10/2016,Administrative Miscellaneous / Initial Inspection
369237,41077301,COZY CAFE,MANHATTAN,43,EAST 1 STREET,10003,2124750177,Middle Eastern,03/09/2016,Violations were cited in the following area(s).,15I,''''No Smoking and/or 'Smoking Permitted sig...,Not Critical,NaN,NaN,NaN,06/10/2016,Smoke-Free Air Act / Initial Inspection
386974,41453018,ROCK BAR,MANHATTAN,185,CHRISTOPHER STREET,10014,2126751864,American,02/11/2015,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,23.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection
444524,41453018,ROCK BAR,MANHATTAN,185,CHRISTOPHER STREET,10014,2126751864,American,02/11/2015,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,23.0,NaN,NaN,06/10/2016,Cycle Inspection / Initial Inspection


In [38]:
# TODO:
# remove dups

In [17]:
all_data['SCORE'].describe()

count    421286.000000
mean         19.499426
std          13.090040
min          -2.000000
25%                NaN
50%                NaN
75%                NaN
max         156.000000
Name: SCORE, dtype: float64

In [31]:
# http://stackoverflow.com/questions/14451185/better-binning-in-pandas
deciles = pd.qcut(all_data['SCORE'], 10)
counts = pd.value_counts(deciles)

counts.reindex(deciles.cat.categories)

[-2, 8]      50293
(8, 10]      48375
(10, 12]     63971
(12, 13]     26604
(13, 16]     23524
(16, 19]     40402
(19, 23]     48102
(23, 27]     38170
(27, 36]     41934
(36, 156]    39911
Name: SCORE, dtype: int64

In [37]:
(float(len(all_data[all_data['SCORE'] > 50])) / len(all_data)) * 100

3.136826040469736

looks like there might be an outlier in the scores given that one of the bins is (36, 156], the stddev is 13.09 and ~ 3% of the SCORES are more than 50. These might just be egregious cases though. Otherwise, looks pretty good.

This looks like an interesting data set with 450806 inspection results. The possibility of exploring categorical and numerical trends exists. Let's start with understanding how many inspections took place in each burough.

## How Many Inspections In Each Burough

In [67]:
all_data[['CAMIS', 'BORO']].groupby(['BORO']).count().add_prefix('count_').sort_values(by='count_CAMIS')

,count_CAMIS
BORO,
Missing,26
STATEN ISLAND,14785
BRONX,40384
QUEENS,106906
BROOKLYN,108691
MANHATTAN,180014


This shows the number of inspection records in each burough. However, there can be multiple records for each individual inspection that takes place. To get at the number of unique inspections, the data would have to be grouped by CAMIS and date of inspection (assuming only one inspection at each restaurant can occur).

## Who Are The Worst Critical Offenders

### Who has the most number of critical violations?

In [84]:
worst_offenders = all_data[all_data['CRITICAL FLAG'] == 'Critical']
#len(worst_offenders)
sorted_worst_offenders = worst_offenders[['CAMIS', 'CRITICAL FLAG']].groupby(['CAMIS']).count().sort_values(by='CRITICAL FLAG', ascending=False)
sorted_worst_offenders.head(10)


,CRITICAL FLAG
CAMIS,
41586091,66
41602559,64
41683816,60
41510846,58
41197758,57
41653529,56
41583748,56
41459659,56
41289382,56
